In [16]:
from model.upscale_model import UpsampleModule
from torchvision.transforms import Compose,ToTensor, ToPILImage, Resize
from omegaconf import OmegaConf
import numpy as np
import cv2
from PIL import Image
import torch
import os

def train_lr_transform(crop_size, upscale_factor, images):
    transform = Compose([
        ToPILImage(),
        Resize(crop_size // upscale_factor, interpolation=Image.BICUBIC),
        ToTensor()
    ])
    for i in range(len(images)):
        img = cv2.resize(images[i], (256, 256))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = transform(img)
        images[i] = torch.tensor(img)
    images = torch.stack(images)
    return images

def save_images(path, flag, images):
    for i in range(len(images)):
        img = images[i].detach().numpy().transpose((1,2,0))
        img = Image.fromarray((img * 255).astype(np.uint8))
        filename = f'restore_{flag}_{i}.png'
        new_path = os.path.join(path, filename)
        img.save(new_path)

if __name__ == "__main__":
    # load conf
    confdir = "/share/program/dxs/RSISR/configs/model.yaml"
    conf = OmegaConf.load(confdir)
    # load model
    upsample = UpsampleModule(conf.upsample)
    test_images = []
    test_img1 = '/share/program/dxs/RSISR/test_demo/airplane08.png'
    test_img2 = '/share/program/dxs/RSISR/test_demo/freeway26.png'
    save_path = "/share/program/dxs/RSISR/test_demo/test_module/"
    test_img1 = Image.open(test_img1).convert("RGB")
    test_img1 = np.array(test_img1)
    test_images.append(test_img1)
    test_img2 = Image.open(test_img2).convert("RGB")
    test_img2 = np.array(test_img2)
    test_images.append(test_img2)
    test_tensors = train_lr_transform(256, 4, test_images)
    save_images(save_path, 'lr', test_tensors)
    output_features = upsample(test_tensors)
    save_images(save_path, 'upsample', output_features)

/tmp/ipykernel_2871532/2991566931.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images[i] = torch.tensor(img)


In [ ]:
from PIL import Image

# 函数：获取图片分辨率
def get_image_resolution(image_path):
    try:
        # 打开图片
        with Image.open(image_path) as img:
            # 获取分辨率
            width, height = img.size
            return width, height
    except Exception as e:
        print(f"无法打开图片: {e}")
        return None

# 主函数
if __name__ == "__main__":
    # 输入图片路径
    image_path = '/share/program/dxs/Database/data/DIV2K_test_HR/0801.png'
    resolution = get_image_resolution(image_path)

    if resolution:
        print(f"图片分辨率: {resolution[0]} x {resolution[1]}")

torch.Size([3, 64, 64])
torch.Size([3, 64, 64])


/home/work/anaconda3/envs/daixingshuo_rsisr/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/tmp/ipykernel_27078/1289130466.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images[i] = torch.tensor(img)


[1, 2, 3]
